In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np
# from convert import *

In [3]:
from fxpmath import Fxp
import numpy as np

class Converter():

    def encode(self, input_data, signed=True, total_bits=16, fractional_bits=13):
        '''

        Converts input data from float/int python data types to ap_fixed with total bits and fractional_bits and returns its uint32 equivalent

        :param input_data: can be both a single int/float number or a numpy array
        :param signed: Boolean, if the input data is signed or not
        :param total_bits: numer of total bits used to represent the input data
        :param fractional_bits: number of fractional bits used to represent the input data. Integer bits = total bits - fractional bits
        :return: input data converted to uint32 format. 0.5 can be represented with 4 bits as 0.100. This is converted into 0100 (fractional point removed) and then converted to int.
                 0.5 as input is converted to 4 as uint32.

        '''
        fixed_point_representation = Fxp(input_data, signed=signed, n_word = total_bits, n_frac = fractional_bits)
        uint_coverted = np.uint32(fixed_point_representation.uraw())
        return uint_coverted



    def decode(self, input_data, total_bits=16, fractional_bits=13):
        '''
        Converts input data from uint32 format to float with total_bits and fractional_bits resolution

        :param input_data: can be both a single int/float number or a numpy array
        :param total_bits: numer of total bits used to represent the input data
        :param fractional_bits: number of fractional bits used to represent the input data. Integer bits = total bits - fractional bits
        :return: converted input data from uint32 to float
        '''

        if type(input_data) is not np.ndarray:
            input_data = np.array([input_data])

        #Function taken from here: https://discuss.pynq.io/t/how-to-use-ap-fixed-data-type-to-communicate-with-the-ip-made-by-the-vivado-hls/679/5
        condition = 1 << (total_bits - 1)
        mask = (~((1 << total_bits) - 1)) & 0xFFFFFFFF
        return np.where(input_data < condition, input_data, (input_data.view('u4') | mask).view('i4')) / (1 << fractional_bits)





In [13]:
converter = Converter()
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [14]:
idx=np.random.permutation(x_train.shape[0])[:10000]

x_train=x_train[idx]
y_train=y_train[idx]

In [15]:
x_train=x_train/255.0
x_test=x_test/255.0

In [7]:
# x_train=np.array(x_train,np.dtype('u8'))
# x_test=np.array(x_test,np.dtype('u8'))

# y_train=np.array(y_train,np.dtype('u8'))
# y_test=np.array(y_test,np.dtype('u8'))

print(x_train[0])
print(y_train.shape)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

In [16]:
x_train=x_train.reshape(x_train.shape[0],784)
x_test=x_test.reshape(x_test.shape[0],784)

x_train_encoded=np.zeros((x_train.shape[0],784))
x_test_encoded=np.zeros((x_test.shape[0],784))

In [17]:
np.save('x_train_plain.npy',x_train)

In [8]:
for i in range(x_train.shape[0]):
    for j in range(784):
        if x_train[i][j] != 0:
            x_train_encoded[i][j]=int(converter.encode(x_train[i][j]))

# for i in range(x_test.shape[0]):
#     for j in range(784):
#         x_test_encoded[i][j]=int(converter.encode(x_train[i][j]))

In [18]:

np.save("x_train.npy",x_train_encoded)
# np.save("x_test.npy",x_test)
np.save("y_train.npy",y_train)
# np.save("y_test.npy",y_test)


In [10]:
f=open("x_train.txt","w")
x_train= np.reshape(x_train,(x_train.shape[0],-1))

for i in range(x_train.shape[0]):
    
    for j in range(x_train.shape[1]):
        # print(x_train[i][j],end=" ",file=f)
        f.write(f"{x_train[i][j]}"+" ")
    f.write('\n')
f.close()


In [11]:
f=open("y_train.txt","w")
y_train= np.reshape(y_train,(y_train.shape[0],-1))

for i in range(y_train.shape[0]):
    f.write(str(y_train[i][0])+" ")
    f.write('\n')
f.close()

In [ ]:
g=open("test.txt","w")
g.write(f"{0.123}")
g.close()